#Task
- create an auto encoder
(3x32x32) --> (16x16x16) --> (32x8x8) --> (64xx4x4) --> (32x8x8) -->(16x16x16) (3x32x32)

In [ ]:
import torchvision
import torchvision.transforms as transforms
# import libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset,SubsetRandomSampler
from sklearn.model_selection import train_test_split

# for getting summary info on models
from torchsummary import summary

import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')
device = torch.device('cude' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transforms)
testset = torchvision.datasets.CIFAR10(root='./data',
                                        train=False,
                                        download=True,
                                        transform=transforms)


In [ ]:
# transformations
transform = T.Compose([ T.ToTensor(),
                        T.Normalize([.5,.5,.5],[.5,.5,.5])
                       ])

# import the data and simultaneously apply the transform
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,  download=True, transform=transform)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# transform to dataloaders
batchsize    = 32
train_loader = DataLoader(trainset,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader  = DataLoader(testset, batch_size=len(testset))

In [ ]:
# create a class for the model
def makeTheNet():

  class emnistnet(nn.Module):
    def __init__(self):
      super().__init__()
      # encoding layer
      self.enc = nn.Sequential(
          nn.Conv2d(
              in_channels=3,
              out_channels=16,
              kernel_size=4,
              padding=1
              stride=2),
          nn.ReLU(),
          nn.MaxPool2d(2,2),
          nn.Conv2d(6,4,3,padding=1),
          nn.ReLU(),
          nn.MaxPool2d(2,2)
          )

      # decoding layer
      self.dec = nn.Sequential(
          nn.ConvTranspose2d(4,6,3,2),
          nn.ReLU(),
          nn.ConvTranspose2d(6,1,3,2),
          )



    def forward(self,x):
      return self.dec(self.enc(x))

  # create the model instance
  net = emnistnet()

  # loss function
  lossfun = nn.CrossEntropyLoss()

  # optimizer
  optimizer = torch.optim.Adam(net.parameters(),lr=.001)

  return net,lossfun,optimizer

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def trainTheModel(net, lossfun, optimizer, train_loader, test_loader):
  numepochs = 15


  # net.to(device)

  # initialize losses
  trainLoss = torch.zeros(numepochs)
  valLoss  = torch.zeros(numepochs)
  trainAcc  = torch.zeros(numepochs)
  valAcc   = torch.zeros(numepochs)


  # loop over epochs
  for epochi in range(numepochs):

    # loop over training data batches
    net.train()
    batchLoss = []
    batchAcc  = []
    for X,y in train_loader:

      # push data to GPU
      X = X.to(device)
      y = y.to(device)

      # forward pass and loss
      yHat = net(X)
      loss = lossfun(yHat,y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss and Accuracy from this batch
      batchLoss.append(loss.item())
      batchAcc.append( torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
    # end of batch loop...

    # and get average losses and Accuracy rates across the batches
    trainLoss[epochi] = np.mean(batchLoss)
    trainAcc[epochi]  = 100*np.mean(batchAcc)


    net.eval()
    # test performance
    valBatchLoss = []
    valBatchAcc  = []
    for X,y in val_loader: # extract X,y from test dataloader

    # push data to GPU
      X = X.to(device)
      y = y.to(device)

      with torch.no_grad(): # deactivates autograd
        yHat = net(X)
        loss = lossfun(yHat,y)

      valBatchLoss.append(loss.item())
      valBatchAcc.append( torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
    # get loss and Accor rate from the test batch
    valLoss[epochi] = loss.item()
    valAcc[epochi]  = 100*torch.mean((torch.argmax(yHat,axis=1) == y).float()).item()

  # end epochs

  # function output
  return trainLoss,valLoss,trainAcc,valAcc,net


In [ ]:
net, lossfun, optimizer = makeTheNet()
trainLoss,valLoss,trainAcc,valAcc,net = trainTheModel(net, lossfun, optimizer, train_loader, test_loader)

In [ ]:
net.eval()
X,y = next(iter(test_loader))
X = X.to(device)
y = y.to(device)
with torch.no_grad():
  yHat = net(X)
  loss = lossfun(yHat, y)

testLoss = loss.item()
testAcc  = 100*torch.mean((torch.argmax(yHat,axis=1) == y).float()).item()

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(5,6))


ax[0].plot(trainLoss,'s-',label='Train')
ax[0].plot(valLoss,'o-',label='Val')
ax[0].plot(testLoss,'o-',label='Val')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss (CEL)')
ax[0].set_title('Model loss')

In [ ]:
fig,axs = plt.subplots(2,10,figsize=(14,4))
yHat.cpu()
for i in range(10):
  #Model output
  pic = yHat[i,:,:,:].detach().numpy().transpose((1,2,0))
  pic = pic/2 + .5 # undo normalization
  axs[0,i].imshow(pic)
  axs[0,i].set_title(f'[ {np.min(pic):.2f}, {np.max(pic):.2f} ]',fontsize=10)
  axs[0,i].axis('off')

  # origional images
  pic = X[i,:,:,:].detach().numpy().transpose((1,2,0))
  pic = pic/2 + .5 # undo normalization
  axs[1,i].imshow(pic)
  axs[1,i].set_title(f'[ {np.min(pic):.2f}, {np.max(pic):.2f} ]',fontsize=10)
  axs[1,i].axis('off')


plt.show()